In [1]:

from pyspark.sql import SparkSession

#Crear sesión
spark = SparkSession.builder.appName("ProyectoMetroRegresionLineal").master("local[*]").config("spark.sql.session.timeZone","America/Santiago").getOrCreate()

print("App Nombre: ",spark.sparkContext.appName)

App Nombre:  ProyectoMetroRegresionLineal


In [2]:
# Carga de datos en un DF de Spark

ruta_datos = "metro_santiago_200.csv"

df = (
    spark.read.option("header", True).
    option("inferSchema", True).
    csv(ruta_datos)
)

# Pequeña prueba para ver si los datos se cargaron en el DF
df.printSchema()

root
 |-- id_evento: integer (nullable = true)
 |-- linea: string (nullable = true)
 |-- estacion: string (nullable = true)
 |-- tipo_incidente: string (nullable = true)
 |-- duracion_minutos: integer (nullable = true)
 |-- hora: timestamp (nullable = true)



In [4]:
# Exploración breve del DF

from pyspark.sql import functions as F

# Filas y Columnas
print("Filas: ",df.count(), "| Columnas: ",len(df.columns))

Filas:  200 | Columnas:  6


In [5]:
#Conteo nulos
nulos = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])
nulos.show()

+---------+-----+--------+--------------+----------------+----+
|id_evento|linea|estacion|tipo_incidente|duracion_minutos|hora|
+---------+-----+--------+--------------+----------------+----+
|        0|    0|       0|             0|               0|   0|
+---------+-----+--------+--------------+----------------+----+



In [18]:
# Preparación de los datos para el ML regresión lineal
from pyspark.ml.feature import StringIndexer, VectorAssembler

# 1) Extraer la hora como número
df = df.withColumn("hora_num", F.hour("hora"))

# 2) Definir los indexadores no transformados
indexer_linea = StringIndexer(inputCol="linea", outputCol="linea_index")
indexer_estacion = StringIndexer(inputCol="estacion", outputCol="estacion_index")
indexer_tipo = StringIndexer(inputCol="tipo_incidente",outputCol="tipo_index")

# 3) Ajustar (entrenar) los modelos de indexación
indexer_linea_model = indexer_linea.fit(df)
indexer_estacion_model = indexer_estacion.fit(df)
indexer_tipo_model = indexer_tipo.fit(df)

# 4) Transformar el DF con estos modelos
df = indexer_linea_model.transform(df)
df = indexer_estacion_model.transform(df)
df = indexer_tipo_model.transform(df)

# 5) Ensamblar las columnas [X]
assembler = VectorAssembler (
    inputCols=["linea_index","estacion_index", "tipo_index","hora_num"], #Son aquellas columnas que asumen el rol de X
    outputCol= "features"
)

df = assembler.transform(df)

# 6) Definir [y] label = duracion_minutos
df = df.withColumnRenamed("duracion_minutos","label")

# 7) Vista al DF
df.select("linea","estacion","tipo_incidente","hora","features","label").show(10, truncate=False)

Exception ignored in: <function JavaWrapper.__del__ at 0x7a3606f439c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
                                              ^^^^^^^^^^^^^^
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


+-----+--------------+------------------------+-------------------+-------------------+-----+
|linea|estacion      |tipo_incidente          |hora               |features           |label|
+-----+--------------+------------------------+-------------------+-------------------+-----+
|L1   |Baquedano     |Falla eléctrica         |2025-08-29 07:30:00|(4,[3],[7.0])      |20   |
|L2   |Cal y Canto   |Mantenimiento           |2025-08-29 22:15:00|[1.0,17.0,1.0,22.0]|15   |
|L3   |Plaza de Armas|Evacuación              |2025-08-29 14:45:00|[2.0,15.0,2.0,14.0]|10   |
|L4   |Las Torres    |Interrupción de servicio|2025-08-29 09:15:00|[5.0,18.0,3.0,9.0] |25   |
|L5   |Santa Ana     |Falla eléctrica         |2025-08-29 16:30:00|[3.0,6.0,0.0,16.0] |30   |
|L6   |Los Leones    |Retraso                 |2025-08-29 11:00:00|[4.0,4.0,4.0,11.0] |15   |
|L4A  |Ñuñoa         |Mantenimiento           |2025-08-29 06:45:00|[6.0,8.0,1.0,6.0]  |20   |
|L1   |Los Héroes    |Falla eléctrica         |2025-08-29 18

In [19]:
# Entrenar nuestro modelo y evaluarlo con algunas métricas = regresión lineal
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# 1) Entrenamiento = Train / Test
train_df, test_df = df.select("features","label").randomSplit([0.7,0.3], seed=42)

# 2) Definir el modelo ML y entrenarlo
lr = LinearRegression(featuresCol="features",labelCol = "label")
lr_model = lr.fit(train_df)

# 3) Predicciones con test
preds = lr_model.transform(test_df)
preds.select("prediction","label").show(10, truncate = False)

+------------------+-----+
|prediction        |label|
+------------------+-----+
|22.00958223069215 |15   |
|21.34642025409477 |20   |
|23.3400124971093  |25   |
|23.113535852969562|25   |
|22.34468477577364 |25   |
|21.264060206223892|15   |
|23.59413575428753 |20   |
|22.8829349122672  |20   |
|21.06091261481182 |15   |
|22.631748455432177|25   |
+------------------+-----+
only showing top 10 rows



In [20]:
# 4) Métricas de evaluación
r2 = RegressionEvaluator(metricName = "r2", labelCol= "label", predictionCol="prediction").evaluate(preds)
print(f"R2 = {r2:.2f}")

R2 = -0.02


# Actividad

In [27]:
# Carga de datos en un DF de Spark

ruta_datos = "metro_santiago_200.csv"

df = (
    spark.read.option("header", True).
    option("inferSchema", True).
    csv(ruta_datos)
)

# Pequeña prueba para ver si los datos se cargaron en el DF
df.printSchema()

root
 |-- id_evento: integer (nullable = true)
 |-- linea: string (nullable = true)
 |-- estacion: string (nullable = true)
 |-- tipo_incidente: string (nullable = true)
 |-- duracion_minutos: integer (nullable = true)
 |-- hora: timestamp (nullable = true)



In [10]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

categorical_cols = ["linea", "estacion", "tipo_incidente"]
output_cols = [col + "_index" for col in categorical_cols]

# 1) Extraer la hora como número
df = df.withColumn("hora_num", F.hour("hora"))

# 2) Definir los indexadores no transformados
string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=output_cols)

# 3) One Hot Encoder

one_hot_encoder = OneHotEncoder(inputCols=string_indexer.getOutputCols(), outputCols=[col + "_ohe" for col in categorical_cols])

# 4)
numerical_cols = ["hora_num"]

# 5) Assemble
assembler_inputs = numerical_cols + one_hot_encoder.getOutputCols()
vector_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="unscaled_features")

# 6) Escalar
scaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

# 7) Regresion Lineal
lr = LinearRegression(featuresCol="features", labelCol="label")

#  8) Pipeline
pipeline = Pipeline(stages=[string_indexer, one_hot_encoder, vector_assembler, scaler, lr])

# 9) Entrenar
(training_data, test_data) = df.randomSplit([0.8, 0.2], seed=42)

model = pipeline.fit(training_data)

predictions = model.transform(test_data)

print("\n--- Predicciones en el conjunto de prueba ---")
predictions.select("prediction","label").show()

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

IllegalArgumentException: label does not exist. Available: id_evento, linea, estacion, tipo_incidente, duracion_minutos, hora, hora_num, linea_index, estacion_index, tipo_incidente_index, linea_ohe, estacion_ohe, tipo_incidente_ohe, unscaled_features, features